In this notebook, we have loaded both train and test dataset and filled the missing values.

for Education we used "Missing" to fill

for previous_year_rating we used -1 to fill the values for both train and test data set.

Also previous_year rating must be related to length of years of service. for those employees where lenght of service is 1, those people should not be having the previous year rating. so this would be the critical feature to determine the promotion.

obviously people having no experience can not be promoted.

region is converted in to codes as that contains 34 unique values and creating 34 one hot encoded values does not make sense. but this can be tried out as it is possible that promotion are based on region wise.

Department we will one hot encode as its common basic people working in HR will be promoted in same department.

Region is having almost similar distribution in both training and test set.

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics

#### Load Training and Test Set

In [ ]:
df_train = pd.read_csv(r'C:\Users\GJ\Desktop\Machine Learning\fastai\WNS\train.csv', low_memory=False)
df_test = pd.read_csv(r'C:\Users\GJ\Desktop\Machine Learning\fastai\WNS\test.csv', low_memory=False)

#### Drop columns like region, age, gender and recruitment channel. as deciding the promotion should not be biased by these parameters.

In [ ]:
df_train.drop(columns=['region', 'age', 'gender', 'recruitment_channel'], axis=0, inplace=True)
df_test.drop(columns=['region', 'age', 'gender', 'recruitment_channel'], axis=0, inplace=True)

#### Treat Missing values and convert year rating to category

In [ ]:
df_train.education.fillna(value="Missing", inplace=True)
df_test.education.fillna(value="Missing", inplace=True)

In [ ]:
df_train.previous_year_rating.fillna(value=0.0, inplace=True)
df_test.previous_year_rating.fillna(value=0.0, inplace=True)

In [ ]:
df_train.previous_year_rating = df_train.previous_year_rating.astype('category')
df_test.previous_year_rating = df_test.previous_year_rating.astype('category')

In [ ]:
#Finance, HR, Legal and R&D
#df_train.department[df_train.department=='Finance'] = 'Others'
#df_train.department[df_train.department=='HR'] = 'Others'
#df_train.department[df_train.department=='Legal'] = 'Others'
#df_train.department[df_train.department=='R&D'] = 'Others'

In [ ]:
#df_test.department[df_test.department=='Finance'] = 'Others'
#df_test.department[df_test.department=='HR'] = 'Others'
#df_test.department[df_test.department=='Legal'] = 'Others'
#df_test.department[df_test.department=='R&D'] = 'Others'

### One hot encoding of categorical variables

Getting dummies for categorical variables. Also here we are treating rating as categorical variable since it has missing vallues which seems to be related to employees who are 1 year experienced.

In [8]:
df_train = pd.concat([df_train.drop('department', axis=1), pd.get_dummies(df_train.department, prefix='department')], axis=1)
df_test = pd.concat([df_test.drop('department', axis=1), pd.get_dummies(df_test.department, prefix='department')], axis=1)

In [9]:
df_train = pd.concat([df_train.drop('education', axis=1), pd.get_dummies(df_train.education, prefix='education')], axis=1)
df_test = pd.concat([df_test.drop('education', axis=1), pd.get_dummies(df_test.education, prefix='education')], axis=1)

In [10]:
df_train = pd.concat([df_train.drop('previous_year_rating', axis=1), pd.get_dummies(df_train.previous_year_rating, prefix='previous_year_rating')], axis=1)
df_test = pd.concat([df_test.drop('previous_year_rating', axis=1), pd.get_dummies(df_test.previous_year_rating, prefix='previous_year_rating')], axis=1)

In [11]:
df_train.is_promoted.value_counts()

0    50140
1     4668
Name: is_promoted, dtype: int64

We can try for training score, like should we take log or not.

Creating 2 new features for awards_won & training score and KPI & training score.
    This combination has way more weightage for minority class as compared to majority class in this case.

In [ ]:
#award_train = (df_train['awards_won?']==1) & (df_train['avg_training_score']>80)
#award_test = (df_test['awards_won?']==1) & (df_test['avg_training_score']>80)
#len(award_train), len(award_test)

In [ ]:
#KPI_train = (df_train['KPIs_met >80%']==1) & (df_train['avg_training_score']>80)
#KPI_test = (df_test['KPIs_met >80%']==1) & (df_test['avg_training_score']>80)
#len(KPI_train), len(KPI_test)

In [ ]:
#df_train['award_training80'] = np.where(award_train, '1', '0' )
#df_test['award_training80'] = np.where(award_test, '1', '0' )

In [ ]:
#df_train['KPI_training80'] = np.where(KPI_train, '1', '0' )
#df_test['KPI_training80'] = np.where(KPI_test, '1', '0' )

### Normalizing the variables

In [12]:
df_train.avg_training_score = np.log(df_train.avg_training_score)
df_test.avg_training_score = np.log(df_test.avg_training_score)

In [13]:
no_trng_min = df_train.no_of_trainings.min()
no_trng_max = df_train.no_of_trainings.max()
avg_trng_score_mean = df_train.avg_training_score.mean()
avg_trng_score_std = df_train.avg_training_score.std()
length_service_min = df_train.length_of_service.min()
length_service_max = df_train.length_of_service.max()

In [14]:
df_train.no_of_trainings = (df_train.no_of_trainings - no_trng_min)/(no_trng_max)
df_test.no_of_trainings = (df_test.no_of_trainings - no_trng_min)/(no_trng_max)

In [15]:
df_train.avg_training_score = (df_train.avg_training_score - avg_trng_score_mean)/(avg_trng_score_std)
df_test.avg_training_score = (df_test.avg_training_score - avg_trng_score_mean)/(avg_trng_score_std)

In [16]:
df_train.length_of_service = (df_train.length_of_service - length_service_min)/(length_service_max)
df_test.length_of_service = (df_test.length_of_service - length_service_min)/(length_service_max)

In [17]:
df_train.head()

,employee_id,no_of_trainings,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Analytics,department_Finance,department_HR,...,education_Bachelor's,education_Below Secondary,education_Master's & above,education_Missing,previous_year_rating_0.0,previous_year_rating_1.0,previous_year_rating_2.0,previous_year_rating_3.0,previous_year_rating_4.0,previous_year_rating_5.0
0,65438,0.0,0.189189,1,0,-1.137576,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,65141,0.0,0.081081,0,0,-0.160248,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,7513,0.0,0.162162,0,0,-1.040083,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,2542,0.1,0.243243,0,0,-1.040083,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,48945,0.0,0.027027,0,0,0.786150,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [18]:
#os.makedirs('tmp', exist_ok=True)
df_train.to_feather(r'tmp\train_v9_0')
df_test.to_feather(r'tmp\test_v9_0')